<a href="https://colab.research.google.com/github/kb22/Tensorflow-Course/blob/master/Classification_of_Fashion_MNIST_using_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification of Fashion MNIST using ANN

In this notebook, I'll use an Artificial Neural Network to train on the Fashion MNIST dataset and then classify amongst 10 different clothing items including shirt, sandel etc.

## Background

The Fashion MNIST dataset includes many images belonging to 10 different classes of clothing. These classes are:
- T-shirt/top
- Trouser
- Pullover
- Dress
- Coat
- Sandal
- Shirt
- Sneaker
- Bag
- Ankle boot

The dataset allows us to use deep learning models to train on the images and then use it to classify new set of images from these classes. Each image is of the size 28x28 pixels. Each pixel value is represented as a value from 0 to 255. 0 refers to black, 255 refers to white and the number in between refer to different shades of gray. All images are grayscale.

## Import libraries

I'll import tensorflow and other libraries. The Fashion MNIST dataset is available in `tensorflow_datatsets`.

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import tensorflow_datasets as tfds

I'll also set the logging to only log Tensorflow errors.

In [0]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

## Data retrival

The `tensorflow_datasets` already has the MNIST dataset. So, the best way to get the data is from here itself.
The `tfds.load()` method is used to fetch the dataset.
- **name:** It is the name of the dataset that we want to download.
- **data_dir:** It defines the directory where we want to save/read the dataset to/from.
- **as_supervised:** It returns the labels along with the features so that we can perform supervised learning.
- **with_info:** It returns information about the dataset.

The complete list of datasets is available [here](https://www.tensorflow.org/datasets/datasets).

In [0]:
dataset, info = tfds.load(name = 'fashion_mnist', data_dir = 'data', as_supervised = True, with_info = True)

Let's first take a look at the information available with this dataset.

In [17]:
print("Dataset name: {}".format(info.name))
print("Dataset version: {}".format(info.version))
print("Description: {}".format(info.description))

Dataset name: fashion_mnist
Dataset version: 1.0.0
Description: Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.


There are 60,000 train images and 10,000 test images. Let's get those train and test data into separate variables.

In [0]:
train_dataset = dataset['train']
test_dataset = dataset['test']